In [ ]:
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import ast
import json
import re
nltk.download('punkt')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # Load data from .npy file
data = np.load("/content/drive/MyDrive/BrainTeaser/word_puzzle.npy", allow_pickle=True)[()]

questions = [entry['question'] for entry in data]
options = [entry['choice_list'] for entry in data]  # List of option lists
correct_indices = [entry['label'] for entry in data]
# data

# Rest of the code remains mostly the same

In [ ]:
# file_path = '/content/csqa_rs_train.jsonl'  # Replace with the path to your JSON Lines file

# # Read the entire file into a list using a list comprehension
# with open(file_path, 'r', encoding='utf-8') as file:
#     data = [json.loads(line) for line in file]
# questions = [entry['question']['stem'] for entry in data]

# options = []
# for entry in data:
#     choices = [choice['text'] for choice in entry['question']['choices']]
#     options.append(choices)

# correct_indices = [ord(entry['answerKey']) - ord('A') +1 for entry in data]
# print(data[0])
# print(questions[0])
# print(options[0])
# print(correct_indices)


In [ ]:
# import re

# def clean_text(text):
#     # Remove extra spaces
#     text = re.sub(r'\s+', ' ', text)  # Replace consecutive spaces with a single space

#     # Remove backslashes
#     text = text.replace('\n', '')

#     return text

# # Example text with extra spaces and backslashes
# example_text = "This is some     example    text with \\ extra spaces and backslashes.    "

# # Clean the text
# cleaned_text = clean_text(example_text)

# # Print the cleaned text
# print(cleaned_text)

In [ ]:
# df = pd.DataFrame(data)
# print(df.shape)

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(questions)
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
# Convert correct answer indices to one-hot encoding
num_options = len(options[0])  # Assuming all questions have the same number of options
encoded_correct_answers = tf.one_hot(correct_indices, num_options)



In [ ]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_options, activation='softmax'))  # Number of output units equals the number of options

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, encoded_correct_answers, epochs=10, validation_split=0.1)


Epoch 1/10
14/14 [==============================] - 6s 234ms/step - loss: 1.3269 - accuracy: 0.3348 - val_loss: 1.2270 - val_accuracy: 0.3600
Epoch 2/10
14/14 [==============================] - 4s 266ms/step - loss: 1.2249 - accuracy: 0.3529 - val_loss: 1.1240 - val_accuracy: 0.3600
Epoch 3/10
14/14 [==============================] - 2s 134ms/step - loss: 1.2192 - accuracy: 0.3009 - val_loss: 1.1398 - val_accuracy: 0.3600
Epoch 4/10
14/14 [==============================] - 2s 127ms/step - loss: 1.1910 - accuracy: 0.3529 - val_loss: 1.1279 - val_accuracy: 0.3600
Epoch 5/10
14/14 [==============================] - 1s 96ms/step - loss: 1.1719 - accuracy: 0.3575 - val_loss: 1.1078 - val_accuracy: 0.3600
Epoch 6/10
14/14 [==============================] - 1s 68ms/step - loss: 1.1690 - accuracy: 0.3213 - val_loss: 1.1260 - val_accuracy: 0.3600
Epoch 7/10
14/14 [==============================] - 1s 55ms/step - loss: 1.1118 - accuracy: 0.3688 - val_loss: 1.1155 - val_accuracy: 0.3600
Epoch 8/1

In [ ]:
# Example inference for a question and options from the dataset
example_index = 2  # You can change this index to select a specific example from your dataset
given_question = [questions[example_index]]
given_options = options[example_index]

# Tokenize and pad the given question
given_question_sequence = tokenizer.texts_to_sequences(given_question)
given_padded_sequence = pad_sequences(given_question_sequence, maxlen=max_sequence_length, padding='post')

# Predict the option probabilities for the given question
predicted_option_probs = model.predict(given_padded_sequence)

# Get the index of the predicted correct option
predicted_option_index = np.argmax(predicted_option_probs)
predicted_option = given_options[predicted_option_index]

# Print the predicted option and its probability
print("Given Question:", given_question[0])
print("Given Options:", given_options)
print("Predicted Option Index:", predicted_option_index)
print("Predicted Correct Option:", predicted_option)
print("Correct Index:", correct_indices[example_index])

1/1 [==============================] - 0s 343ms/step
Given Question: How do you spell COB in seven letters?
Given Options: ['COBCOBB', 'SEE O BEE', 'COBBLER', 'None of above.']
Predicted Option Index: 0
Predicted Correct Option: COBCOBB
Correct Index: 1


In [ ]:
import numpy as np
import tensorflow as tf
import csv
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import ast
import json
import re

# Load data from .npy file
data = np.load("/content/drive/MyDrive/BrainTeaser/word_puzzle.npy", allow_pickle=True)[()]

questions = [entry['question'] for entry in data]
options = [entry['choice_list'] for entry in data]  # List of option lists
correct_indices = [entry['label'] for entry in data]

# Text preprocessing
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and numbers using regex
    # text = re.sub(r'[^a-z\s]', '', text)
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stop words or perform additional text cleaning if necessary
    # Example:
    # words = [word for word in words if word not in stopwords]
    return ' '.join(words)

# Preprocess questions
preprocessed_questions = [preprocess_text(question) for question in questions]

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_questions)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(preprocessed_questions)
max_sequence_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')

# Convert correct answer indices to one-hot encoding
num_options = 4  # Assuming each question has 4 options
encoded_correct_answers = tf.one_hot(correct_indices, num_options)

# Build and train the model (as in previous code)
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_options, activation='softmax'))  # Number of output units equals the number of options

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, encoded_correct_answers, epochs=10, validation_split=0.1)

# Example inference for a question and options from the dataset
example_index = 0  # You can change this index to select a specific example from your dataset
given_question = [preprocess_text(questions[example_index])]
given_options = options[example_index]

# Tokenize and pad the given question
given_question_sequence = tokenizer.texts_to_sequences(given_question)
given_padded_sequence = pad_sequences(given_question_sequence, maxlen=max_sequence_length, padding='post')

# Predict the option probabilities for the given question
predicted_option_probs = model.predict(given_padded_sequence)

# Get the index of the predicted correct option
predicted_option_index = np.argmax(predicted_option_probs)
predicted_option = given_options[predicted_option_index]

# Print the predicted option, its probability, and the correct index
print("Given Question:", given_question[0])
print("Given Options:", given_options)
print("Predicted Option Index:", predicted_option_index)
print("Predicted Correct Option:", predicted_option)
print("Correct Index:", correct_indices[example_index])


Epoch 1/10
14/14 [==============================] - 6s 205ms/step - loss: 1.2821 - accuracy: 0.2919 - val_loss: 1.1119 - val_accuracy: 0.3800
Epoch 2/10
14/14 [==============================] - 7s 507ms/step - loss: 1.2072 - accuracy: 0.3710 - val_loss: 1.1467 - val_accuracy: 0.3600
Epoch 3/10
14/14 [==============================] - 4s 308ms/step - loss: 1.2009 - accuracy: 0.3145 - val_loss: 1.1260 - val_accuracy: 0.3600
Epoch 4/10
14/14 [==============================] - 4s 274ms/step - loss: 1.1877 - accuracy: 0.3439 - val_loss: 1.1279 - val_accuracy: 0.3800
Epoch 5/10
14/14 [==============================] - 2s 185ms/step - loss: 1.1790 - accuracy: 0.3190 - val_loss: 1.1288 - val_accuracy: 0.3600
Epoch 6/10
14/14 [==============================] - 2s 153ms/step - loss: 1.1213 - accuracy: 0.3710 - val_loss: 1.0940 - val_accuracy: 0.3800
Epoch 7/10
14/14 [==============================] - 2s 151ms/step - loss: 1.1124 - accuracy: 0.3529 - val_loss: 1.1259 - val_accuracy: 0.2600
Epoch 

In [ ]:
encoded_correct_answers = np.array(correct_indices)  # Convert to numpy array
encoded_correct_answers = np.expand_dims(encoded_correct_answers, axis=-1)  # Expand dimensions to match logits

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=200, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(256, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)))  # Added dropout
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))  # Added dropout
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_options, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
model.fit(padded_sequences, encoded_correct_answers, epochs=20, batch_size=64, validation_split=0.2)


Epoch 1/20
7/7 [==============================] - 22s 1s/step - loss: 1.3045 - accuracy: 0.3079 - val_loss: 1.1972 - val_accuracy: 0.3333
Epoch 2/20
7/7 [==============================] - 7s 985ms/step - loss: 1.2359 - accuracy: 0.3537 - val_loss: 1.1512 - val_accuracy: 0.3434
Epoch 3/20
7/7 [==============================] - 10s 2s/step - loss: 1.2382 - accuracy: 0.3079 - val_loss: 1.1526 - val_accuracy: 0.3434
Epoch 4/20
7/7 [==============================] - 7s 957ms/step - loss: 1.1703 - accuracy: 0.3232 - val_loss: 1.1235 - val_accuracy: 0.3333
Epoch 5/20
7/7 [==============================] - 10s 1s/step - loss: 1.1860 - accuracy: 0.3613 - val_loss: 1.1431 - val_accuracy: 0.3333
Epoch 6/20
7/7 [==============================] - 9s 1s/step - loss: 1.1213 - accuracy: 0.3791 - val_loss: 1.1300 - val_accuracy: 0.3434
Epoch 7/20
7/7 [==============================] - 9s 1s/step - loss: 1.0538 - accuracy: 0.4682 - val_loss: 1.1348 - val_accuracy: 0.3434
Epoch 8/20
7/7 [================

In [ ]:
# Example inference for a question and options from the dataset
example_index = 0  # You can change this index to select a specific example from your dataset
given_question = [questions[example_index]]
given_options = options[example_index]

# Tokenize and pad the given question
given_question_sequence = tokenizer.texts_to_sequences(given_question)
given_padded_sequence = pad_sequences(given_question_sequence, maxlen=max_sequence_length, padding='post')

# Predict the option probabilities for the given question
predicted_option_probs = model.predict(given_padded_sequence)

# Get the index of the predicted correct option
predicted_option_index = np.argmax(predicted_option_probs)
predicted_option = given_options[predicted_option_index]

# Print the predicted option and its probability
print("Given Question:", given_question[0])
print("Given Options:", given_options)
print("Predicted Option Index:", predicted_option_index)
print("Predicted Correct Option:", predicted_option)
print("Correct Index:", correct_indices[example_index])

1/1 [==============================] - 2s 2s/step
Given Question: How do you spell COW in thirteen letters?
Given Options: ['SEE O DOUBLE YOU.', 'SEE OH DEREFORD', 'COWCOWCOWCOWW', 'None of above.']
Predicted Option Index: 0
Predicted Correct Option: SEE O DOUBLE YOU.
Correct Index: 0
